In [1]:
import os
import shap
import pandas as pd
import numpy as np
import time
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import leaguegamefinder
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
import sqlite3 as sq

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
with sq.connect('./basnya.db') as con:
    df = pd.read_sql("select * from boxscoresummaryv2_0", con=con)
print(df.shape)
df.head()

(3495, 16)


,index,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,WH_STATUS,GAME_ID_STR
0,0,2021-10-03T00:00:00,1,12100001,3,Final,20211003/BKNLAL,1610612747,1610612751,2021,4,None,NBA TV,Q4 - NBA TV,1,0012100001
1,0,2021-10-04T00:00:00,1,12100002,3,Final,20211004/PHITOR,1610612761,1610612755,2021,4,,None,Q4 -,1,0012100002
2,0,2021-10-04T00:00:00,2,12100003,3,Final,20211004/ORLBOS,1610612738,1610612753,2021,4,,None,Q4 -,1,0012100003
3,0,2021-10-04T00:00:00,3,12100004,3,Final,20211004/ATLMIA,1610612748,1610612737,2021,4,,NBA TV,Q4 - NBA TV,1,0012100004
4,0,2021-10-04T00:00:00,4,12100005,3,Final,20211004/NOPMIN,1610612750,1610612740,2021,4,,None,Q4 -,1,0012100005


In [3]:
df.groupby('SEASON').GAME_ID.nunique()

SEASON
2020    1363
2021    2087
Name: GAME_ID, dtype: int64

In [4]:
def get_game_ids(team_id=None):
    _dfs = []
    for season in ['2020-21', '2021-22']:
        _dfs.append(leaguegamefinder.LeagueGameFinder(
            team_id_nullable=team_id, 
            season_nullable=season, 
            season_type_nullable=leaguegamefinder.SeasonTypeNullable.regular,
            league_id_nullable=leaguegamefinder.LeagueIDNullable.nba)
                    .get_data_frames()[0])
    return pd.concat(_dfs, axis=0)

TEAM_ID = None
# TEAM_ID = teams.find_team_by_abbreviation('DAL')['id']

game_ids = get_game_ids(team_id=TEAM_ID)
print(game_ids.shape)
GAME_ID_TO_DATE_MAP = game_ids.set_index('GAME_ID').GAME_DATE.to_dict()
game_ids.head()

(4620, 28)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22020,1610612738,BOS,Boston Celtics,0022001073,2021-05-16,BOS @ NYK,L,240,92,35,86,0.407,11,41,0.268,11,15,0.733,9,37,46,19,4,6,12,19,-4.0
1,22020,1610612752,NYK,New York Knicks,0022001073,2021-05-16,NYK vs. BOS,W,238,96,34,79,0.430,13,27,0.481,15,19,0.789,4,37,41,16,7,6,10,17,4.0
2,22020,1610612763,MEM,Memphis Grizzlies,0022001070,2021-05-16,MEM @ GSW,L,241,101,40,92,0.435,6,25,0.240,15,19,0.789,12,27,39,25,12,5,12,21,-12.0
3,22020,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16,TOR vs. IND,L,240,113,39,97,0.402,12,38,0.316,23,29,0.793,16,32,48,21,8,3,12,12,-12.0
4,22020,1610612737,ATL,Atlanta Hawks,0022001066,2021-05-16,ATL vs. HOU,W,240,124,49,90,0.544,7,21,0.333,19,26,0.731,11,45,56,32,10,8,13,16,29.0


In [5]:
game_ids.groupby('SEASON_ID').GAME_ID.nunique()#.value_counts(dropna=False)

SEASON_ID
22020    1080
22021    1230
Name: GAME_ID, dtype: int64

In [6]:
players_df = pd.DataFrame(players.get_players())
print(players_df.shape)
players_df.head()

(4815, 5)


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


In [7]:
with sq.connect('./basnya.db') as con:
    game_ids.to_sql(name='games', con=con)
    players_df.to_sql(name='players', con=con)